In [ ]:
'''
a. News article title
b. News article subtitle
c. News article date
d. News article author
e. Media contact name
f. Media contact phone number
g. Media contact email address
'''

In [7]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [20]:
url = 'https://newsroom.unsw.edu.au'
req_ses = requests.Session()
req = req_ses.get(url=url)
req.raise_for_status()
req.encoding = req.apparent_encoding
html = req.text
bs = BeautifulSoup(html,"html.parser")
html

'<!DOCTYPE html>\n<!--[if lt IE 7]>      <html class="no-js lt-ie9 lt-ie8 lt-ie7"> <![endif]-->\n<!--[if IE 7]>         <html class="no-js lt-ie9 lt-ie8"> <![endif]-->\n<!--[if IE 8]>         <html class="no-js lt-ie9"> <![endif]-->\n<!--[if gt IE 8]><!--> <html class="no-js"> <!--<![endif]-->\n<head profile="http://www.w3.org/1999/xhtml/vocab">\n  <meta http-equiv="X-UA-Compatible" content="IE=edge">\n  <meta name="viewport" content="width=device-width, initial-scale=1">\n  <title>UNSW Newsroom</title>\n  <meta http-equiv="Content-Type" content="text/html; charset=utf-8" />\n<link rel="shortcut icon" href="https://newsroom.unsw.edu.au/profiles/newsroom/themes/custom/carto/favicon.ico" type="image/vnd.microsoft.icon" />\n<script type="text/javascript">dataLayer = [{"articleTitle":"UNSW innovators honoured by the Australian Academy of Technology and Engineering","articleURL":"https://newsroom.unsw.edu.au/news/science-tech/unsw-innovators-honoured-australian-academy-technology-and-engine

In [14]:
def try_get_text(bs):
    if bs:
        return bs.get_text().strip()
    else:
        return

In [22]:
nodes = bs.find_all('div', attrs={'class':'node-article'})
data = []
for node in nodes:
    data_node = {}
    title_html = node.find('div', attrs={'class':'field-title-field'})
    if title_html:
        title_url = title_html.find('a')
        data_node['title'] = title_url.get_text()
        node_url = url + title_url.get("href")
        data_node['url'] = node_url
        
    data_node['sub_title'] = try_get_text(node.find('div', attrs={'class':'field-standfirst'}))
    data_node['post_date'] = try_get_text(node.find('div', attrs={'class':'field-post-date'}))

    # Inside Node
    req = req_ses.get(url=node_url)
    req.raise_for_status()
    req.encoding = req.apparent_encoding
    node_html = req.text
    node_bs = BeautifulSoup(node_html,"html.parser")
    data_node['author'] = try_get_text(node_bs.find('div', attrs={'class':'field-byline-term'}))
    media_contact = node_bs.find('div', attrs={'id':'block-views-media-contact-teaser-media-contact'})
    if media_contact: 
        data_node['contact_name'] = try_get_text(media_contact.find('div', attrs={'class':'field-contact-name'}))
        data_node['contact_phone'] = try_get_text(media_contact.find('div', attrs={'class':'field-contact-phone'}))
        data_node['contact_email'] = try_get_text(media_contact.find('div', attrs={'class':'field-contact-email'}))
    else:
        media_contact = node_bs.find('div', attrs={'class':'pane-media-contact-cognitives'})
        if media_contact:
            data_node['contact_name'] = try_get_text(media_contact.find('div', attrs={'class':'field-media-contact-name'}))
            # contact_phone = media_contact.find('div', attrs={'class':'field-contact-phone'})
            data_node['contact_email'] = try_get_text(media_contact.find('div', attrs={'class':'field-media-contact-email'}))
        else:
            pass
    data.append(data_node)
pd.DataFrame(data)

,title,url,sub_title,post_date,author,contact_name,contact_email,contact_phone
0,UNSW innovators honoured by the Australian Aca...,https://newsroom.unsw.edu.au/news/science-tech...,UNSW academics are named Fellows for their wor...,26 Nov 2021,Con Anagnostopoulos,Con Anagnostopoulos,c.anagnostopoulos@unsw.edu.au,NaN
1,Online supervision will support mental health ...,https://newsroom.unsw.edu.au/news/health/onlin...,UNSW Sydney researchers have been awarded a $1...,26 Nov 2021,Yolande Hutchinson,Yolande Hutchinson,y.hutchinson@unsw.edu.au,0420 845 023
2,What is orthokeratology? And will it help slow...,https://newsroom.unsw.edu.au/news/health/what-...,Orthokeratology involves wearing a specially d...,26 Nov 2021,Paul Gifford,NaN,NaN,NaN
3,"Vital Signs: cautious on rates, strong on clim...",https://newsroom.unsw.edu.au/news/business-law...,None,26 Nov 2021,Richard Holden,NaN,NaN,NaN
4,"Country, culture and nature-based solutions ke...",https://newsroom.unsw.edu.au/news/science-tech...,None,25 Nov 2021,UNSW Canberra,Rachel Packham,r.packham@adfa.edu.au,0423 800 109
5,Take the stress out of Christmas by being kind...,https://newsroom.unsw.edu.au/news/health/take-...,Christmas can be stressful for a variety of re...,25 Nov 2021,Emi Berry,Emi Berry,e.berry@unsw.edu.au,0413 803 873
6,La Niña has been declared. Why should we care?,https://newsroom.unsw.edu.au/news/science-tech...,"With La Niña prolonging the wet season, there ...",24 Nov 2021,"Andrea Taschetto, Agus Santoso",Diane Nazaroff,diane.nazaroff@unsw.edu.au,0424479199
7,Alec Baldwin: was it criminal negligence in th...,https://newsroom.unsw.edu.au/news/business-law...,Homicide law and ballistics experts discuss wh...,24 Nov 2021,Dawn Lo,Dawn Lo,dawn.lo@unsw.edu.au,+61433018180
8,UNSW economist receives prestigious Paul Bourk...,https://newsroom.unsw.edu.au/news/general/unsw...,Tim Neal has been recognised for his insights ...,24 Nov 2021,Belinda Henwood,Belinda Henwood,b.henwood@unsw.edu.au,NaN
9,Will Australia follow Europe into a fourth COV...,https://newsroom.unsw.edu.au/news/health/will-...,"For starters, we need to start thinking of ful...",23 Nov 2021,Raina Macintyre,NaN,NaN,NaN


In [23]:
pd.DataFrame(data).fillna('')

,title,url,sub_title,post_date,author,contact_name,contact_email,contact_phone
0,UNSW innovators honoured by the Australian Aca...,https://newsroom.unsw.edu.au/news/science-tech...,UNSW academics are named Fellows for their wor...,26 Nov 2021,Con Anagnostopoulos,Con Anagnostopoulos,c.anagnostopoulos@unsw.edu.au,
1,Online supervision will support mental health ...,https://newsroom.unsw.edu.au/news/health/onlin...,UNSW Sydney researchers have been awarded a $1...,26 Nov 2021,Yolande Hutchinson,Yolande Hutchinson,y.hutchinson@unsw.edu.au,0420 845 023
2,What is orthokeratology? And will it help slow...,https://newsroom.unsw.edu.au/news/health/what-...,Orthokeratology involves wearing a specially d...,26 Nov 2021,Paul Gifford,,,
3,"Vital Signs: cautious on rates, strong on clim...",https://newsroom.unsw.edu.au/news/business-law...,,26 Nov 2021,Richard Holden,,,
4,"Country, culture and nature-based solutions ke...",https://newsroom.unsw.edu.au/news/science-tech...,,25 Nov 2021,UNSW Canberra,Rachel Packham,r.packham@adfa.edu.au,0423 800 109
5,Take the stress out of Christmas by being kind...,https://newsroom.unsw.edu.au/news/health/take-...,Christmas can be stressful for a variety of re...,25 Nov 2021,Emi Berry,Emi Berry,e.berry@unsw.edu.au,0413 803 873
6,La Niña has been declared. Why should we care?,https://newsroom.unsw.edu.au/news/science-tech...,"With La Niña prolonging the wet season, there ...",24 Nov 2021,"Andrea Taschetto, Agus Santoso",Diane Nazaroff,diane.nazaroff@unsw.edu.au,0424479199
7,Alec Baldwin: was it criminal negligence in th...,https://newsroom.unsw.edu.au/news/business-law...,Homicide law and ballistics experts discuss wh...,24 Nov 2021,Dawn Lo,Dawn Lo,dawn.lo@unsw.edu.au,+61433018180
8,UNSW economist receives prestigious Paul Bourk...,https://newsroom.unsw.edu.au/news/general/unsw...,Tim Neal has been recognised for his insights ...,24 Nov 2021,Belinda Henwood,Belinda Henwood,b.henwood@unsw.edu.au,
9,Will Australia follow Europe into a fourth COV...,https://newsroom.unsw.edu.au/news/health/will-...,"For starters, we need to start thinking of ful...",23 Nov 2021,Raina Macintyre,,,


In [21]:
pd.DataFrame(data).to_csv('Data.csv')